## Ce que j'ai fait pour tout installer :

1. J'ai intealler postgres (https://www.postgresql.org/download/windows/)
2. J'ai créer la database en superuser (ouais c'est pas bien mais flemme vu que dans ts les cas vous allez devoir resetup les rôles :angel:)  
`CREATE DATABASE "sens-art";`
1. J'ai appliqué le code si dessous :

# Todo TESTS

## Pbs de sauvegardes (prioritaire)
+ sauvegarde des fichiers images
+ Sauvegarde des fichiers sons
+ Sauvegarde des hasmap (labels) (osef)

## fonctions a mettre en place prioritaires
+ Bash setup database

## fonction.alité.s a mettre en place après
+ Quand on crée un objet vérifier s'il est déjà dans la BDD
+ diverses sécurités

In [1]:
#pip install psycopg2-binary

import psycopg2
import sys

# Pour tout le monde


# Pour Test et intégration
ourHost = "localhost" # L'hôte auquel on ce connecte
ourPort = "5432"      # Le port auquel on ce connecte, par défaut
ourDatabase = "sens-art"

In [2]:
def connect():
    user = "postgres"
    pwd = "PACT25"
    conn = psycopg2.connect(database = ourDatabase, user = user, password = pwd, host = ourHost, port = ourPort)
    cur = conn.cursor()
    return conn, cur

def close(conn, cur):
    conn.commit()
    cur.close()
    conn.close()

In [3]:
class BDD(): 
    def __enter__(self): 
        self.conn, self.cur = connect()
        return self.cur
      
    def __exit__(self, exc_type, exc_value, exc_traceback): 
        close(self.conn, self.cur)

# init()

with BDD() as cur:
    cur.execute("""SELECT * FROM work WHERE title = 'La Joconde';""")
    print(cur.fetchone())
    

In [4]:
def init():
    with BDD() as cur :
        
        cur.execute("""CREATE TABLE artist (
            id integer PRIMARY KEY,
            name varchar(60),
            movement varchar(60),
            complete bool
        );""")

        cur.execute("""CREATE TABLE work (
            id integer PRIMARY KEY,
            title varchar(60), 
            artist integer REFERENCES artist,
            date varchar(10),
            image bytea,
            text text,
            label text,
            complete bool
        );""")

        cur.execute("""CREATE TABLE script (
            id integer REFERENCES work,
            matrixES text,
            nameSoundAtmosphere varchar(60),
            matrixSoundAtmosphere text,
            soundEffectFile bytea,
            complete bool
        );""")

In [5]:
### Stolen sur https://zetcode.com/python/psycopg2/ + ctflF(Image)

def readImage(pathImage):

    fin = None

    try:
        fin = open(pathImage, "rb")
        img = fin.read()
        return img

    except IOError as e:

        print(f'Error {e.args[0]}, {e.args[1]}')
        sys.exit(1)

    finally:

        if fin:
            fin.close()
            

def writeImage(data, name):

    fout = None

    try:
        fout = open(name, 'wb')
        fout.write(data)

    except IOError as e:

        print(f"Error {0}")
        sys.exit(1)

    finally:

        if fout:
            fout.close()

In [15]:
def getWork(ID):
    with BDD() as cur:
        cur.execute("""SELECT id FROM work WHERE id = %s;""", (ID,))
        if not cur.fetchone():
            return None
        a = Work(None, None, None, None, None)
        cur.execute("""SELECT * FROM work WHERE id = %s;""", (ID,))
        a.ID, a.title, a.artist, a.date, a.imgData, a.txt, a.labels, a.complete = cur.fetchone()
    a.img = a.title + "Image"
    writeImage(a.imgData, a.img)
    a.artist = getArtist(a.artist)
    return a
        

def getArtist(ID):
    with BDD() as cur:
        cur.execute("""SELECT id FROM artist WHERE id = %s;""", (ID,))
        if not cur.fetchone():
            return None
        a = Artist(None, None)
        cur.execute("""SELECT * FROM artist WHERE id = %s;""", (ID,))
        a.ID, a.name, a.movement, a.complete = cur.fetchone()
    return a

In [7]:
class Work:
    def __init__(self, title, date, img, txt, artist):
        self.title = title
        self.img = img
        if img:
            self.imgData = readImage(self.img)
        self.date = date
        self.txt = txt
        self.artist = artist

    ID = None #ID unique de l'oeuvre
    title = None #string
    artist = None #objet artist
    date = None  #string au format AAAA/SS, avec des chiffres arabes, mettre "X" si inconnu (exemple pour une oeuvre du XIIIeme dont on ne connais pas la date exacte : XXXX/13)
    img = None #nomDuFicbier
    imgData = None # donnée pour python 
    txt = None #le texte en temps que tel
    labels = None #hashmap de labal-score (inutilisé)
    complete = False #booletan l'objet est-il complet ?    
    

    def isFull(self):
        if(self.labels != None):
            self.complete = True
        return self.complete
    
    def createID(self):
        if(self.ID == None):
            with BDD() as cur:
                cur.execute("""SELECT EXISTS(SELECT 1 FROM work WHERE title = %s);""", (self.title,))
                if(cur.fetchone()[0]):
                    cur.execute("""SELECT id FROM work WHERE title = %s;""", (self.title,))
                    self.ID = cur.fetchone()[0]
                else:
                    cur.execute("""SELECT MAX(id) FROM work;""")
                    IDmax = cur.fetchone()[0]
                    if IDmax:
                        self.ID = cur.fetchone()[0] + 1
                    else:
                        self.ID = 0
    
    def getScript(self):
        a = Script(self)
        with BDD() as cur:
            cur.execute("""SELECT EXISTS(SELECT 1 FROM script WHERE id = %s)""",
                       (self.ID,))
            if(cur.fetchone()[0]):
                cur.execute("""SELECT * FROM script WHERE id = %s""",
                           (self.ID, ))
                a.ID, a.matrixES, a.nameSoundAtmosphere, a.matrixSoundAtmosphere, a.soundEffectFile, a.complete = cur.fetchone()
        return a
            
    
    def push(self):
        self.isFull()
        self.createID()
        self.artist.push()
        with BDD() as cur:
            cur.execute("""SELECT EXISTS(SELECT id FROM work WHERE id = %s)""", (self.ID,))
            if(cur.fetchone()[0]):
                cur.execute("""DELETE FROM work WHERE id = %s;""", (self.ID,))
            cur.execute("""INSERT INTO work (id, title, artist, date, image, text, label, complete) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s); """,
                (self.ID, self.title, self.artist.ID, self.date, psycopg2.Binary(self.imgData), self.txt, self.labels, self.complete))

In [8]:
class Artist:
    def __init__(self, name, movement):
        self.name = name
        self.movement = movement
        
    name = None #Nom d'artiste, format String
    movement = None #nom du mouvement, format String
        ID = None #ID unique de l'artiste
    complete = True #booletan l'objet est-il complet ? (oui dès qu'il est créé)
    
    def isFull(self):
        return self.complete
    
    def createID(self):
        if(self.ID == None):
            with BDD() as cur:
                cur.execute("""SELECT EXISTS(SELECT 1 FROM artist WHERE name = %s);""", (self.name,))
                if(cur.fetchone()[0]):
                    cur.execute("""SELECT id FROM artist WHERE name = %s;""", (self.name,))
                    self.ID = cur.fetchone()[0]
                else:
                    cur.execute("""SELECT MAX(id) FROM artist;""")
                    IDmax = cur.fetchone()[0]
                    if IDmax:
                        self.ID = cur.fetchone()[0] + 1
                    else:
                        self.ID = 0

    def push(self):
        self.isFull()
        self.createID()
        with BDD() as cur:
            cur.execute("""SELECT EXISTS(SELECT id FROM artist WHERE id = %s)""", (self.ID,))
            if(cur.fetchone()[0]):
                cur.execute("""DELETE FROM artist WHERE id = %s;""", (self.ID,))
            cur.execute("""INSERT INTO artist (id, name, movement, complete)  
                VALUES (%s, %s, %s, %s); """,
                (self.ID, self.name, self.movement, self.complete))

In [9]:
class Script:
    def __init__(self, work):
        self.work = work
        self.ID = self.work.ID
    
    matrixES = None
    nameSoundAtmosphere = None
    matrixSoundAtmosphere = None
    soundEffectFile = None
    work = None #objet work
    ID = None #ID unique de l'oeuvre
    complete = False #booletan l'objet est-il complet ?
    


    def isFull(self):
        if (none in [self.matrixES, self.nameSoundAtmosphere, self.matrixSoundAtmosphere, self.soundEffectFile, self.work]):
            self.complete = True
        return self.complete
    
    def createID(self):
        if(self.ID == None):
            self.work.createID()
            self.ID = self.work.ID

    def push(self):
        self.isFull()
        self.work.push()
        self.createID()
        with BDD() as cur:
            cur.execute("""SELECT EXISTS(SELECT id FROM script WHERE id = %s)""", (self.ID,))
            if(cur.fetchone()[0]):
                cur.execute("""DELETE FROM script WHERE id = %s;""", (self.ID,))
            cur.execute("""INSERT INTO script (id, matrixES, nameSoundAtmosphere, soundEffectFile, complete) 
                VALUES (%s, %s, %s, %s, %s); """,
                (self.ID, self.matrixES, self.nameSoundAtmosphere, self.soundEffectFile, self.complete))

In [10]:
class SoundEffects:
    def __init__(self, name, soundFile):
        self.name = name
        self.soundFile = soundFile
        self.ID = 42
    
    name = None #String
    soundFile = None #Type du fichier ? path ou fichier ?
    ID = None #ID unique de l'oeuvre
    complete = True#booletan l'objet est-il complet ? (oui dès qu'il est créé)
    
    
    
    def isFull(self):
        return self.complete
    
    def createID(self):
        
    def push(self):
        

### Tests :

init()



artist = Artist("Da Vinci", "Haute Renaissance")

text = "La Joconde, ou Portrait de Mona Lisa, est un tableau de l'artiste Léonard de Vinci, réalisé entre 1503 et 1506 ou entre 1513 et 15161,2, et peut-être jusqu'à 1519 (l'artiste étant mort cette année-là, le 2 mai)3, qui représente un portrait mi-corps, probablement celui de la Florentine Lisa Gherardini, épouse de Francesco del Giocondo. Acquise par François Ier, cette peinture à l'huile sur panneau de bois de peuplier de 77 × 53 cm est exposée au musée du Louvre à Paris. La Joconde est l'un des rares tableaux attribués de façon certaine à Léonard de Vinci. La Joconde est devenue un tableau éminemment célèbre car, depuis sa réalisation, nombre d'artistes l'ont pris comme référence. À l'époque romantique, les artistes ont été fascinés par ce tableau et ont contribué à développer le mythe qui l'entoure, en faisant de ce tableau l’une des œuvres d'art les plus célèbres du monde, si ce n'est la plus célèbre : elle est en tout cas considérée comme l'une des représentations d'un visage féminin les plus célèbres au monde4. Au XXIe siècle, elle est devenue l'objet d'art le plus visité au monde, devant le diamant Hope5, avec 20 000 visiteurs qui viennent l'admirer et la photographier quotidiennement6. "
work = Work("La Joconde", "1519", "La Joconde.jpg", text, artist)

work.push()


work2 = getWork(0)

In [62]:
def getAllWork():
    with BDD() as cur:
        cur.execute("""SELECT MAX(id) FROM work""")
        maxID = cur.fetchone()[0]
        listWork = []
        if maxID != None:
            for i in range(maxID+1):
                temp = getWork(i)
                if(temp != None):
                    listWork.append(temp)
    return listWork